In [1]:
from surprise import SVD, Dataset, accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/arti/.surprise_data/ml-100k


In [2]:
algo = SVD()
algo.fit(trainset)

In [3]:
predictions = algo.test(testset)
predictions[:5]

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.723055675355707, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8891336812229964, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.231657208454199, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5806544974739456, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.416277872278764, details={'was_impossible': False})]

In [4]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.723055675355707),
 ('882', '291', 3.8891336812229964),
 ('535', '507', 4.231657208454199)]

In [5]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
pred

Prediction(uid='196', iid='302', r_ui=None, est=4.194015585158004, details={'was_impossible': False})

In [6]:
accuracy.rmse(predictions)

RMSE: 0.9489


0.9488718756680571

In [8]:
import pandas as pd
import os

HOME = os.path.expanduser('~')

ratings = pd.read_csv(HOME + '/Downloads/ratings.csv')
ratings.to_csv(HOME + '/Downloads/ratings_noh.csv', index=False, header=False)

In [10]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file(HOME + '/Downloads/ratings_noh.csv', reader=reader)

In [11]:
algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9457


0.9456994032241722

In [12]:
reader = Reader(rating_scale=(.5, 5.))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [13]:
from surprise.model_selection import cross_validate

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8815  0.8631  0.8675  0.8756  0.8712  0.8718  0.0064  
MAE (testset)     0.6778  0.6678  0.6649  0.6724  0.6676  0.6701  0.0045  
Fit time          3.69    3.74    3.77    3.65    3.70    3.71    0.04    
Test time         0.10    0.10    0.19    0.19    0.11    0.14    0.04    


{'test_rmse': array([0.8815467 , 0.86305738, 0.867464  , 0.87561443, 0.87124796]),
 'test_mae': array([0.677781  , 0.6678035 , 0.66491973, 0.67242486, 0.66757193]),
 'fit_time': (3.694110155105591,
  3.7363555431365967,
  3.7722442150115967,
  3.6486196517944336,
  3.7022106647491455),
 'test_time': (0.10254073143005371,
  0.10218358039855957,
  0.18980145454406738,
  0.18667006492614746,
  0.10762548446655273)}

In [14]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20,40,60], 'n_factors': [50,100,200]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'], gs.best_params['rmse'])

0.8769382936063265 {'n_epochs': 20, 'n_factors': 50}


In [15]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(.5, 5.))
data_folds = DatasetAutoFolds(ratings_file=HOME + '/Downloads/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [16]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [17]:
movies = pd.read_csv(HOME + '/Downloads/movies.csv')
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print("No ratings")
    
print(movies[movies==42])

No ratings
      movieId title genres
0         NaN   NaN    NaN
1         NaN   NaN    NaN
2         NaN   NaN    NaN
3         NaN   NaN    NaN
4         NaN   NaN    NaN
...       ...   ...    ...
9737      NaN   NaN    NaN
9738      NaN   NaN    NaN
9739      NaN   NaN    NaN
9740      NaN   NaN    NaN
9741      NaN   NaN    NaN

[9742 rows x 3 columns]


In [18]:
uid = str(9)
iid = str(42)
pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [22]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수: ', len(seen_movies), ' 추천 대상 영화수: ', len(unseen_movies),
         ' 전체 영화 수: ', len(total_movies))
    return unseen_movies

In [24]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(id1, title, rating) for id1, title, rating in 
                      zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    return top_movie_preds

In [25]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('### Top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
    print(top_movie[1], ": ", top_movie[2])

평점 매긴 영화 수:  46  추천 대상 영화수:  9696  전체 영화 수:  9742
### Top-10 추천 영화 리스트 ###
Usual Suspects, The (1995) :  4.306302135700814
Star Wars: Episode IV - A New Hope (1977) :  4.281663842987387
Pulp Fiction (1994) :  4.278152632122759
Silence of the Lambs, The (1991) :  4.226073566460876
Godfather, The (1972) :  4.1918097904381995
Streetcar Named Desire, A (1951) :  4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) :  4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) :  4.108009609093436
Goodfellas (1990) :  4.083464936588478
Glory (1989) :  4.07887165526957
